In [ ]:
import os
import matplotlib.pyplot as plt
from osfile.manager import findFile
from slide.tile import SlidePatcher, TileEncoder
from slide.utils import get_slide_reader, print_attrs, read_h5_coords, read_h5_features
import h5py
from slide.load import encoder_factory
import numpy as np
from draw.palette import mosaic

In [ ]:
dir = "/cluster/CBIO/data1/mblons/tipit/data/pathology/image/hes/wsi"
act = "P_AAAA1287_HES"
wsipath, _ = findFile(dir, strings=act, isFolder=False)
print(wsipath)
reader = get_slide_reader(wsipath)
SLIDE = reader(wsipath)
print(f"mpp at level 0: {SLIDE.mpp}")
print(f"magnification: {SLIDE.magnification}")
print(f"level count: {SLIDE.level_count}")
print(f"level dimensions: {SLIDE.level_dimensions}")
print(f"level downsamples {SLIDE.level_downsamples}")
PATCH_SIZE = 256
SOURCE_MAG = SLIDE.magnification
TARGET_MAG = 20
OVERLAP = 0
MASK_DOWNSAMPLE = 32
MASK_TOLERANCE = 0.7
OUTPUT_DIR = f"./1_tutorial/tile_feat_{TARGET_MAG}x_{PATCH_SIZE}px_{OVERLAP}px_overlap"

In [ ]:
thumbnail = SLIDE.get_thumbnail(size=(1024, 1024))
fig, ax = plt.subplots(1, 1, figsize=(10, 10), layout="constrained")
ax.set_axis_off()
ax.imshow(thumbnail, aspect="equal")
plt.show()

In [ ]:
patcher = SlidePatcher(
    SLIDE,
    mag_0=SOURCE_MAG,
    mag_target=TARGET_MAG,
    patch_size=PATCH_SIZE,
    overlap=OVERLAP,
    mask_downsample=MASK_DOWNSAMPLE,
    mask_tolerance=MASK_TOLERANCE,
    xywh_only=False,
    dst=OUTPUT_DIR,
)
print(f"target mag: {patcher.mag_target}")
print(f"level: {patcher.level}")
print(f"mask downsampling: {patcher.mask_downsample}")
print(f"mask level: {patcher.level_mask}")

In [ ]:
with h5py.File(patcher.patch_path, "r") as h5_file:
    print("Contents and Attributes in patch file:")
    h5_file.visititems(print_attrs)

In [ ]:
# visu mask
seg_dir = os.path.join(OUTPUT_DIR, "segmentation")
seg_path = patcher.visualize_tissue_seg(size=(1024, 1024), save_seg=seg_dir, show=True)
print(seg_path)

In [ ]:
# visu cut
visu_dir = os.path.join(OUTPUT_DIR, "visualization")
cut_path = patcher.visualize_cut(size=(1024, 1024), save_cut=visu_dir, show=True)
print(cut_path)

In [ ]:
# get collection of tiles
fig, ax = plt.subplots(1, 1, figsize=(10, 10), layout="constrained")
nt = 36
idx_samples = np.random.choice(patcher.nb_valid_patches, nt, replace=False)
iimg = [patcher.get_tile(*patcher.valid_patches[i]) for i in idx_samples]
ax.set_title(f"Tiles Selection in: {SLIDE.name}")
mosaic(iimg, size=(PATCH_SIZE, PATCH_SIZE), ax=ax)

In [ ]:
ENCODER = "uni_v2"
DEVICE = "cpu"
BATCH_SIZE = 512
model = encoder_factory(ENCODER)
model.to(DEVICE)
encoder = TileEncoder(
    SLIDE,
    tile_encoder=model,
    coords_path=os.path.join(OUTPUT_DIR, "patches", f"{SLIDE.name}_patches.h5"),
    device=DEVICE,
    batch_max=BATCH_SIZE,
    feat_only=False,
    dst=OUTPUT_DIR,
)

In [ ]:
with h5py.File(encoder.feat_path, "r") as h5_file:
    print("Contents and Attributes in feats file:")
    h5_file.visititems(print_attrs)

In [ ]:
feats_attrs, feats = read_h5_features(encoder.feat_path)
coord_attrs, coords = read_h5_coords(encoder.feat_path)

In [ ]:
# visu feat
umap_dir = os.path.join(OUTPUT_DIR, f"cluster_{ENCODER}")
umap_path = encoder.visualize_feat(
    pcs=50, neighbors=20, resolution=0.2, save_cluster=umap_dir, show=True
)
print(umap_path)